In [23]:
import sys
import numpy as np
import cupy as cp

sys.path.append('../')

from mandala.nodecore import Node
from mandala.nodecore import Variable
from mandala.autodiff import autodiff
from mandala.autodiff import layer
from mandala.autodiff import initializers
from mandala import cuda

In [34]:
def linear_forward(x, W, b, xp):
    y = xp.matmul(x, W.T)
    if b is not None:
        y += b
    return y


def linear_backward_W(x, gy, xp):
    gW = xp.matmul(gy.T, x)
    return gW


def linear_backward_x(W, gy, xp):
    gx = np.matmul(gy, W, xp)
    return gx


class LinearFunction(autodiff.AutoDiff):

    def forward(self, xs):
        xp = cuda.get_array_module(xs)
        x, W = xs
        y = Node(linear_forward, [x, W, xp])
        return y

    def backward(self, xs, gy):
        xp = cuda.get_array_module(xs)
        x, W = xs
        gW = Node(linear_backward_W, [x, gy, xp])
        gx = Node(linear_backward_x, [W, gy, xp])
        return gx, gW


class Linear(layer.Layer):
    def __init__(self, in_ch, out_ch, nobias=False,
                 initializer=initializers.HeNormal):
        self.W = Variable(initializer((out_ch, in_ch)))
        if nobias:
            self.b = None
        else:
            self.b = Variable(numpy.zeros(out_ch, dtype=np.float32))

    def __call__(self, x):
        return LinearFunction()([x, self.W])


In [24]:
from chainer import links as L

In [28]:
L.Linear(3, 3).b.shape

(3,)

In [32]:
test = Linear(5, 5)

In [18]:
test.to_gpu()

In [22]:
import numpy


def HeNormal(shape, scale=1.0):
    fan_in = numpy.prod(shape[1:])
    std = scale * numpy.sqrt(2 / fan_in)
    init_W = numpy.random.normal(0, std, shape)
    return init_W.astype(numpy.float32)


In [21]:
numpy.prod((1, 3, 10, 2)[:-1])

30